In [ ]:
'''
Arquivos:
- Estabelecer a forma de guardar os arquivos conforme necessidade da CNN

Funções:
- Função de importação de arquivo com as seguintes colunas:
    Open High Low Close Volume + Indicadores
  Esta mesma função fará os índices das linhas serem as datas
- Função de escolha do intervalo de dias a terem o gráfico montado
- Função que gera o gráfico de candlestick com volume e indicadores

'''

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mplfinance as mpf
from datetime import datetime
from datetime import timedelta
from PIL import Image
from glob import glob


In [2]:
'''
Importa os dados do ativo e renomeia as colunas
'''

asset = 'BTOW3'
time_frame = '1d'
file_name = asset + '_' + time_frame + '.xlsx'
path = './collected_ data/' + file_name

inplace = True
df_input = pd.read_excel(path)
df_input.dropna(inplace = inplace)
df_input.set_index('Data', inplace = inplace)
df_input.sort_index(axis = 0, inplace = inplace)
df_input.rename(inplace = inplace, columns = {
    'Abertura': 'Open',
    'Máxima': 'High',
    'Mínima': 'Low',
    'Fechamento': 'Close',
    'Volume Financeiro': 'Volume',
    'Média Móvel E [9]': 'MME009',
    'Média Móvel A [21]': 'MMA021',
    'Média Móvel A [200]': 'MMA200',
    'Média Móvel A [51]': 'MMA051',
    'Média Móvel A [80]': 'MMA080',
    'Média Móvel A [400]': 'MMA400',
    'Média Móvel E [400]': 'MME400'})

# df_input.head()


In [3]:
'''
Cria reordena as colunas dos indicadores em ordem alfabética
'''

df_candle = df_input[['Open', 'High', 'Low', 'Close', 'Volume']]
df_ind = df_input.reindex(sorted(df_input.columns[5:]), axis = 1)
df = pd.concat([df_candle, df_ind], axis=1)

# df.head()


In [4]:
'''
Verifica se o dataframe reordenado é igual ao original
'''

columns = list(df.columns)
eq = True
for col in columns:
   eq = eq*df[col].equals(df_input[col])
if eq:
    print("Data Frame válido.")
else:
    print("Data Frame inválido!!!!")
    

Data Frame válido.


In [5]:
'''
Aprender a gerar os gráficos para então iterar os intervalos de data
'''

def date_delta(start_date, delta): # Função sem uso por enquanto
    end_date = datetime.strptime(start_date, '%Y-%m-%d') + timedelta(days = delta)
    return end_date.strftime('%Y-%m-%d')


def clean_folder(path, match):
    files_list = glob(path + f'/*{match}*')
    # Escrever loop para deletar os arquivos da lista
    return


def gen_chart(df, date, trade_delta, min_profit, chart_delta, add, save_name):
    end_index = df.index.get_loc(date)
    start_index = end_index - chart_delta
    
    start_trade = df['Open'][end_index + 1]
    end_trade = df['Close'][end_index + trade_delta]
    if end_trade/start_trade >= 1 + min_profit/100:
        trade_class = 'long'
    elif end_trade/start_trade <= 1 - min_profit/100:
        trade_class = 'short'
    else:
        trade_class = 'wait'

    df_plot = df.iloc[start_index:end_index,:]
    erase_color = 'black'

    mc = mpf.make_marketcolors(up = 'g',down = 'r',
                               wick = 'inherit',
                               edge = 'inherit',
                               volume = 'grey')
    s = mpf.make_mpf_style(marketcolors = mc,
                           facecolor = erase_color,
                           edgecolor = erase_color,
                           figcolor = erase_color)
    addplot = mpf.make_addplot(df_plot[add])
    plot = mpf.plot(df_plot, type = 'candle',
                             volume = True,
                             style = s,
                             addplot = addplot,
                             savefig = './generated_data/plot.png')

    uncropped_plot = Image.open('./generated_data/plot.png')
    box = (uncropped_plot.size[0]*0.202,
           uncropped_plot.size[1]*0.11,
           uncropped_plot.size[0]*0.88,
           uncropped_plot.size[1]*0.85)

    file_name = f'./generated_data/{save_name}_{trade_class}.png'
    cropped_plot = uncropped_plot.crop(box).save(file_name)
    return (start_trade, end_trade, trade_class)
    

In [6]:
'''
Chamamento das funções de geração dos gráficos
Implementar o loop for
'''

date_range = pd.date_range(start='2019-12-01', end='2020-04-30')
trade_delta = 3
min_profit = 1 # in %
chart_delta = 22
ind_columns = list(df.columns)[5:9]

for date in date_range:
    try:
        file_date = date.strftime('%Y-%m-%d')
        save_name = f'{asset}/{asset}_{file_date}'
        out = gen_chart(df, date, trade_delta, min_profit, chart_delta, ind_columns, save_name)
        print(f'{file_date} {out[0]} {out[1]} {out[2]}')
    except KeyError:
        continue
    except IndexError:
        continue


2019-12-02 58.7 59.5 long
2019-12-03 59.7 61.41 long
2019-12-04 59.18 60.73 long
2019-12-05 59.26 61.3 long
2019-12-06 61.1 61.4 wait
2019-12-09 60.7 64.61 long
2019-12-10 61.05 66.2 long
2019-12-11 61.4 66.54 long
2019-12-12 64.3 64.28 wait
2019-12-13 67.3 63.88 short
2019-12-16 66.49 63.81 short
2019-12-17 64.39 62.7 short
2019-12-18 63.98 62.6 short
2019-12-19 63.47 62.73 short
2019-12-20 62.7 62.4 wait
2019-12-23 62.86 62.86 wait
2019-12-26 63.0 65.18 long
2019-12-27 62.51 64.3 long
2019-12-30 62.87 63.9 long
2020-01-02 64.44 63.74 short
2020-01-03 64.02 64.59 wait
2020-01-06 64.0 68.04 long
2020-01-07 63.7 66.78 long
2020-01-08 64.83 69.17 long
2020-01-09 68.1 68.35 wait
2020-01-10 66.7 70.11 long
2020-01-13 68.5 70.36 long
2020-01-14 68.0 71.26 long
2020-01-15 70.46 75.68 long
2020-01-16 70.44 76.56 long
2020-01-17 71.2 75.85 long
2020-01-20 75.03 75.35 wait
2020-01-21 77.47 74.25 short
2020-01-22 74.95 72.13 short
2020-01-23 75.05 75.05 wait
2020-01-24 72.2 75.44 long
2020-01-27